In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
def get_label(sample: str) -> str:
    """Gets label of a given sample

    Args:
        sample (str): a filename

    Returns:
        str: label (M or K)
    """
    return sample.split('.')[0][-1]


samples = os.listdir('./data/')
labels = [get_label(sample) for sample in samples]
train = pd.DataFrame({'File' : samples, 'Label' : labels})

In [3]:
import scipy.io.wavfile
def predict(filename: str, male_freqs=[60, 160], female_freqs=[180, 270], iterations=5) -> str:
    """Predicts the gender of a given voice sample

    Args:
        filename (str): sample/
        male_freqs (list, optional): male frequency range. Defaults to [60, 160].
        female_freqs (list, optional): female frequency range. Defaults to [180, 270].
        iterations (int, optinal): number of HPS iterations. Defaults to 5.

    Returns:
        str: prediction ('M' or 'K')
    """
    w, signal = scipy.io.wavfile.read(f'data/{filename}')

    sample_length = len(signal)/w  # length in seconds

    parts = [signal[i*w:(i+1)*w] for i in range(int(sample_length))]  # each part lasts exactly one second

    results = []

    for part in parts:
        hamming_window = np.hamming(len(part))
        data = part*hamming_window
        abs_fft = np.abs(np.fft.fft(data))/w
        fft_r = np.copy(abs_fft)
        for i in range(2, iterations):
            tab = np.copy(abs_fft[::i])
            fft_r = fft_r[:len(tab)]
            fft_r *= tab
        results.append(fft_r)

    result = [0]*len(results[int(len(results)/2)])

    for res in results:
        result += res

    if(sum(result[male_freqs[0]:male_freqs[1]]) > sum(result[female_freqs[0]:female_freqs[1]])): return 'M'
    else: return 'K'

In [5]:
from sklearn.metrics import accuracy_score

y_pred = [predict(file) for file in samples]
accuracy_score(train['Label'], y_pred)

0.945054945054945

In [16]:
def predict_all(dir='./data/', male_freqs=[60, 160], female_freqs=[180, 270], iterations=5):
    global train
    files = os.listdir(dir)
    y_pred = [predict(file, male_freqs, female_freqs, iterations) for file in files]

    return accuracy_score(train['Label'], y_pred)

In [21]:
# grid search
from itertools import product

male_min = np.arange(40, 81, 10)
male_max = np.arange(140, 181, 10)
female_min = np.arange(160, 201, 10)
female_max = np.arange(250, 291, 10)
iterations = np.arange(2, 7, 2)

combinations = list(product(male_min, male_max, female_min, female_max, iterations))
scores = []

for combination in combinations:
    male_min, male_max, female_min, female_max, iterations = combination
    score = predict_all(male_freqs=[male_min, male_max], female_freqs=[female_min, female_max], iterations=iterations)
    scores.append(score)

results = list(zip(combinations, scores))

In [28]:
sorted_results = sorted(results, key=lambda x: x[1], reverse=True)
sorted_results[:15]

[((60, 140, 180, 270, 6), 0.978021978021978),
 ((60, 140, 190, 280, 6), 0.978021978021978),
 ((70, 140, 180, 260, 6), 0.978021978021978),
 ((70, 140, 190, 270, 6), 0.978021978021978),
 ((70, 140, 190, 280, 6), 0.978021978021978),
 ((80, 140, 190, 260, 6), 0.978021978021978),
 ((80, 150, 180, 260, 6), 0.978021978021978),
 ((80, 150, 190, 270, 6), 0.978021978021978),
 ((80, 150, 190, 280, 6), 0.978021978021978),
 ((80, 160, 180, 270, 6), 0.978021978021978),
 ((80, 160, 180, 280, 6), 0.978021978021978),
 ((80, 180, 160, 270, 6), 0.978021978021978),
 ((80, 180, 160, 280, 6), 0.978021978021978),
 ((60, 140, 180, 260, 6), 0.967032967032967),
 ((60, 140, 180, 280, 6), 0.967032967032967)]

In [35]:
predict_all(male_freqs=[80, 160], female_freqs=[180, 280], iterations=6)

0.978021978021978